In [1]:
# ============================================================================
# Notebook: NB_02_Silver_To_Gold
# Proyecto: QAPITAL Real Estate Analytics
# Descripción: Construcción de modelo dimensional (Star Schema)
# Capa: Silver → Gold
# ============================================================================

from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime

# Configuración de paths
base_lakehouse_path = "abfss://WSP_QAPITAL_REALSTATE_PRD@onelake.dfs.fabric.microsoft.com/LKH_QAPITAL_REALSTATE.Lakehouse/"
silver_tables_path = f"{base_lakehouse_path}Tables/"

print("=" * 70)
print("INICIO - Transformación Silver → Gold (Star Schema)")
print(f"Timestamp: {datetime.now()}")
print("=" * 70)

# ============================================================================
# LEER TABLAS SILVER
# ============================================================================

print("\n📂 Cargando tablas Silver...\n")

df_brokers_silver = spark.read.format("delta").load(f"{silver_tables_path}silver_brokers")
df_campaigns_silver = spark.read.format("delta").load(f"{silver_tables_path}silver_campaigns")
df_clients_silver = spark.read.format("delta").load(f"{silver_tables_path}silver_clients")
df_projects_silver = spark.read.format("delta").load(f"{silver_tables_path}silver_projects")
df_properties_silver = spark.read.format("delta").load(f"{silver_tables_path}silver_properties")
df_leads_silver = spark.read.format("delta").load(f"{silver_tables_path}silver_leads")
df_sales_silver = spark.read.format("delta").load(f"{silver_tables_path}silver_sales")

print("✅ 7 tablas Silver cargadas exitosamente")

# ============================================================================
# CREAR DIMENSIONES (Star Schema)
# ============================================================================

print("\n🔨 Creando Dimensiones...\n")

# --- DIM_BROKER ---
dim_broker = df_brokers_silver.select(
    col("BrokerID").alias("broker_key"),
    col("BrokerID").alias("broker_id"),
    col("BrokerName").alias("broker_name"),
    col("Region").alias("broker_region"),
    col("Email").alias("broker_email")
)

dim_broker.write.format("delta").mode("overwrite") \
    .save(f"{silver_tables_path}dim_broker")
print(f"✅ dim_broker: {dim_broker.count()} registros")

# --- DIM_CLIENT ---
dim_client = df_clients_silver.select(
    col("ClientID").alias("client_key"),
    col("ClientID").alias("client_id"),
    col("FirstName").alias("first_name"),
    col("LastName").alias("last_name"),
    concat(col("FirstName"), lit(" "), col("LastName")).alias("full_name"),
    col("Email").alias("email"),
    col("Region").alias("region")
)

dim_client.write.format("delta").mode("overwrite") \
    .save(f"{silver_tables_path}dim_client")
print(f"✅ dim_client: {dim_client.count()} registros")

# --- DIM_PROJECT ---
dim_project = df_projects_silver.select(
    col("ProjectID").alias("project_key"),
    col("ProjectID").alias("project_id"),
    col("ProjectName").alias("project_name"),
    col("City").alias("city"),
    col("Region").alias("region"),
    col("LaunchYear").alias("launch_year"),
    col("Status").alias("status")
)

dim_project.write.format("delta").mode("overwrite") \
    .save(f"{silver_tables_path}dim_project")
print(f"✅ dim_project: {dim_project.count()} registros")

# --- DIM_PROPERTY ---
dim_property = df_properties_silver.select(
    col("PropertyID").alias("property_key"),
    col("PropertyID").alias("property_id"),
    col("ProjectID").alias("project_id"),
    col("PropertyType").alias("property_type"),
    col("Size_m2").alias("size_m2"),
    col("Bedrooms").alias("bedrooms"),
    col("Bathrooms").alias("bathrooms"),
    col("ListPriceUSD").alias("list_price_usd"),
    col("AvailabilityStatus").alias("availability_status")
)

dim_property.write.format("delta").mode("overwrite") \
    .save(f"{silver_tables_path}dim_property")
print(f"✅ dim_property: {dim_property.count()} registros")

# --- DIM_CAMPAIGN ---
dim_campaign = df_campaigns_silver.select(
    col("CampaignID").alias("campaign_key"),
    col("CampaignID").alias("campaign_id"),
    col("CampaignName").alias("campaign_name"),
    col("Channel").alias("channel"),
    col("StartDate").alias("start_date"),
    col("EndDate").alias("end_date"),
    col("BudgetUSD").alias("budget_usd")
)

dim_campaign.write.format("delta").mode("overwrite") \
    .save(f"{silver_tables_path}dim_campaign")
print(f"✅ dim_campaign: {dim_campaign.count()} registros")

# ============================================================================
# CREAR TABLAS DE HECHOS (Fact Tables)
# ============================================================================

print("\n🔨 Creando Tablas de Hechos...\n")

# --- FACT_SALES ---
fact_sales = df_sales_silver.select(
    col("SaleID").alias("sale_id"),
    col("PropertyID").alias("property_key"),
    col("ClientID").alias("client_key"),
    col("BrokerID").alias("broker_key"),
    col("SaleDate").alias("sale_date"),
    year("SaleDate").alias("sale_year"),
    month("SaleDate").alias("sale_month"),
    col("SalePriceUSD").alias("sale_price_usd")
)

fact_sales.write.format("delta").mode("overwrite") \
    .save(f"{silver_tables_path}fact_sales")
print(f"✅ fact_sales: {fact_sales.count()} registros")

# --- FACT_LEADS ---
fact_leads = df_leads_silver.select(
    col("LeadID").alias("lead_id"),
    col("ClientID").alias("client_key"),
    col("PropertyID").alias("property_key"),
    col("CampaignID").alias("campaign_key"),
    col("LeadDate").alias("lead_date"),
    year("LeadDate").alias("lead_year"),
    month("LeadDate").alias("lead_month"),
    col("LeadSource").alias("lead_source")
)

fact_leads.write.format("delta").mode("overwrite") \
    .save(f"{silver_tables_path}fact_leads")
print(f"✅ fact_leads: {fact_leads.count()} registros")

# ============================================================================
# RESUMEN FINAL
# ============================================================================

print("\n" + "=" * 70)
print("✅ MODELO DIMENSIONAL GOLD COMPLETADO")
print("=" * 70)
print("\nDimensiones creadas:")
print("  • dim_broker")
print("  • dim_client")
print("  • dim_project")
print("  • dim_property")
print("  • dim_campaign")
print("\nTablas de Hechos creadas:")
print("  • fact_sales")
print("  • fact_leads")
print(f"\nFinalizado: {datetime.now()}")
print("\n🎉 Capa Gold lista para análisis y visualización")
print("=" * 70)

StatementMeta(, 8ecc1639-cdc7-4b28-98a3-04dd76d5a166, 3, Finished, Available, Finished)

INICIO - Transformación Silver → Gold (Star Schema)
Timestamp: 2025-10-04 18:19:46.888781

📂 Cargando tablas Silver...

✅ 7 tablas Silver cargadas exitosamente

🔨 Creando Dimensiones...

✅ dim_broker: 30 registros
✅ dim_client: 30 registros
✅ dim_project: 30 registros
✅ dim_property: 30 registros
✅ dim_campaign: 30 registros

🔨 Creando Tablas de Hechos...

✅ fact_sales: 30 registros
✅ fact_leads: 30 registros

✅ MODELO DIMENSIONAL GOLD COMPLETADO

Dimensiones creadas:
  • dim_broker
  • dim_client
  • dim_project
  • dim_property
  • dim_campaign

Tablas de Hechos creadas:
  • fact_sales
  • fact_leads

Finalizado: 2025-10-04 18:20:32.631691

🎉 Capa Gold lista para análisis y visualización
